In [1]:
import os
from glob import glob

import dask
import numpy as np
import pandas as pd
import xarray as xr

from utils import city_list, gev_metric_ids
import metric_funcs as mf

### Preliminaries

In [2]:
################
#### Paths #####
################
# Update these for reproduction
from utils import roar_code_path as project_code_path
from utils import roar_data_path as project_data_path

star_path = "/storage/group/pches/default/users/dcl5300/STAR-ESDM/"  # raw STAR-ESDM outputs

In [3]:
##############
### Models ###
##############
ssp245_gcms = np.unique(
    [
        file.split("/")[-1].split(".")[1]
        for file in glob(f"{star_path}/ssp245/*.nc")
    ]
)
ssp585_gcms = np.unique(
    [
        file.split("/")[-1].split(".")[1]
        for file in glob(f"{star_path}/ssp585/*.nc")
    ]
)

if (ssp245_gcms == ssp585_gcms).all():
    gcms = ssp245_gcms
else:
    print("Model mismatch")

In [4]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="20GiB",
    walltime="01:00:00",
)

cluster.scale(jobs=30)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.57:41073,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Calculate metrics

In [5]:
###############################
# Metric calulcation function #
###############################
def calculate_metric(
    metric_func, var_id, gcm, ssp, needed_vars, star_path, out_path
):
    """ 
    For calculating STAR-ESDM metrics.
    """

    # Some preprocessing
    def read_star(file_path):
        _preprocess = lambda ds: ds[[ds.encoding["source"].split("/")[-1].split(".")[3]]
        ]
        ds = xr.open_mfdataset(
            file_path,
            preprocess=_preprocess,
            decode_times=False,
            chunks={"time": 365, "latitude": -1, "longitude": -1},
        )
        ds.time.attrs["calendar"] = "365_day"
        return xr.decode_cf(ds, decode_times=True)

    # Start calculation
    try:
        # Check if done
        if os.path.isfile(out_path):
            print(f"{ssp} {gcm} already done.")
            return None

        # Read
        ds_tmp = xr.merge([read_star(f"{star_path}/{ssp}/downscaled.{gcm}.r1i1p1f1.{var}*") for var in needed_vars])

        # Calculate metric
        ds_out = metric_func(ds_tmp, var_id)

        # Store
        ds_out.to_netcdf(out_path)
        print(f"{ssp} {gcm}")

    # Log if error
    except Exception as e:
        except_path = f"{project_code_path}/code/logs"
        with open(f"{except_path}/metric_calcs/STAR-ESDM_{gcm}_{ssp}_{var_id}.txt", "w") as f:
            f.write(str(e))

In [16]:
%%time
#############
## CDD max ##
#############
var_id = "cdd"
metric_func = mf.calculate_dd_max
needed_vars = ["tasmin", "tasmax"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/max_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC-Earth3-Veg-LR

In [7]:
%%time
#############
## CDD sum ##
#############
var_id = "cdd"
metric_func = mf.calculate_dd_sum
needed_vars = ["tasmin", "tasmax"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/sum_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR already done.
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC

In [8]:
%%time
#############
## HDD sum ##
#############
var_id = "hdd"
metric_func = mf.calculate_dd_sum
needed_vars = ["tasmin", "tasmax"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/sum_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR already done.
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC

In [9]:
%%time
#############
## HDD max ##
#############
var_id = "hdd"
metric_func = mf.calculate_dd_max
needed_vars = ["tasmin", "tasmax"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/max_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR already done.
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC

In [10]:
%%time
#########################
## Average Temperature ##
#########################
var_id = "tas"
metric_func = mf.calculate_avg
needed_vars = ["tasmin", "tasmax"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/avg_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR already done.
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC

In [11]:
%%time
#########################
## Total Precipitation ##
#########################
var_id = "pr"
metric_func = mf.calculate_sum
needed_vars = ["pr"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/sum_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR already done.
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC

In [12]:
%%time
#########################
## Maximum Temperature ##
#########################
var_id = "tasmax"
metric_func = mf.calculate_max
needed_vars = ["tasmax"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/max_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR already done.
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC

In [13]:
%%time
#########################
## Minimum Temperature ##
#########################
var_id = "tasmin"
metric_func = mf.calculate_min
needed_vars = ["tasmin"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/min_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR already done.
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC

In [14]:
%%time
#########################
# Maximum Precipitation #
#########################
var_id = "pr"
metric_func = mf.calculate_max
needed_vars = ["pr"]

out_path = (
    lambda gcm,
    ssp: f"{project_data_path}/metrics/STAR-ESDM/max_{var_id}_{gcm}_r1i1p1f1_{ssp}.nc"
)

# Loop through all
for ssp in ["ssp245", "ssp585"]:
    for gcm in gcms:
        # Calculate metric
        calculate_metric(
            metric_func=metric_func,
            var_id=var_id,
            gcm=gcm,
            ssp=ssp,
            needed_vars=needed_vars,
            star_path=star_path,
            out_path=out_path(gcm, ssp),
        )

ssp245 ACCESS-CM2 already done.
ssp245 ACCESS-ESM1-5 already done.
ssp245 BCC-CSM2-MR already done.
ssp245 CMCC-ESM2 already done.
ssp245 CanESM5 already done.
ssp245 EC-Earth3 already done.
ssp245 EC-Earth3-Veg already done.
ssp245 EC-Earth3-Veg-LR already done.
ssp245 FGOALS-g3 already done.
ssp245 GFDL-CM4 already done.
ssp245 GFDL-ESM4 already done.
ssp245 INM-CM4-8 already done.
ssp245 INM-CM5-0 already done.
ssp245 IPSL-CM6A-LR already done.
ssp245 KACE-1-0-G already done.
ssp245 KIOST-ESM already done.
ssp245 MIROC6 already done.
ssp245 MPI-ESM1-2-HR already done.
ssp245 MPI-ESM1-2-LR already done.
ssp245 MRI-ESM2-0 already done.
ssp245 NESM3 already done.
ssp245 NorESM2-LM already done.
ssp245 NorESM2-MM already done.
ssp245 TaiESM1 already done.
ssp585 ACCESS-CM2 already done.
ssp585 ACCESS-ESM1-5 already done.
ssp585 BCC-CSM2-MR already done.
ssp585 CMCC-ESM2 already done.
ssp585 CanESM5 already done.
ssp585 EC-Earth3 already done.
ssp585 EC-Earth3-Veg already done.
ssp585 EC

## OLD

### Original grid

In [1]:
# # Calculates summary indices for STAR-ESDM ensemble for given SSP
# def get_raw_data(metric_id, ssp, years, lat, lon, out_path, out_str):
#     """
#     Current summary indices calculated: mean, 99th quantile, 99% quantile range
#     `years` define the window over which all outputs are pooled.
#     """

#     def read_and_process(metric_id, gcm, ssp, years, lat, lon):
#         # Read
#         file_path = f"{project_data_path}/metrics/STAR-ESDM/{metric_id}_{gcm}_r1i1p1f1_{ssp}.nc"
#         ds_tmp = xr.open_dataset(file_path)
#         ds_tmp["time"] = ds_tmp["time"].dt.year

#         # Time slice
#         if years is not None:
#             ds_sel = ds_tmp.sel(time=slice(years[0], years[1]))
#         else:
#             ds_sel = ds_tmp.copy()

#         # Location selection
#         if lon < 0:
#             lon = 360 + lon
#         ds_sel = ds_sel.sel(latitude=lat, longitude=lon, method="nearest")

#         # Construct dataframe
#         df_tmp = (
#             ds_sel.to_dataframe()
#             .drop(columns=["latitude", "longitude"])
#             .reset_index()
#         )
#         df_tmp["ssp"] = ssp
#         df_tmp["gcm"] = gcm

#         # Return
#         return df_tmp

#     # Check if done
#     if not os.path.isfile(f"{out_path}/{out_str}.csv"):
#         df_delayed = []
#         # Read all
#         for gcm in gcms:
#             df_tmp = dask.delayed(read_and_process)(
#                 metric_id, gcm, ssp, years, lat, lon
#             )
#             df_delayed.append(df_tmp)

#         # Compute and store
#         df_out = dask.compute(*df_delayed)
#         pd.concat(df_out).to_csv(f"{out_path}/{out_str}.csv", index=False)

In [2]:
# %%time
# # Compute and store timeseries
# ssps = ["ssp245", "ssp585"]

# # Loop through cities
# for city in city_list:
#     lat, lon = city_list[city]
#     # Loop through SSPs
#     for ssp in ssps:
#         # Loop through metrics
#         for metric_id in metric_ids:
#             # Compute
#             get_raw_data(
#                 metric_id=metric_id,
#                 ssp=ssp,
#                 years=None,
#                 lat=lat,
#                 lon=lon,
#                 out_path=f"{project_data_path}/timeseries/original_grids/",
#                 out_str=f"{metric_id}_STAR-ESDM_{ssp}_{city}",
#             )

### OLD

In [1]:
# # Calculates summary indices for STAR-ESDM ensemble for given SSP
# def get_raw_data(metric_id, ssp, years, lat, lon, out_path, out_str):
#     """
#     Current summary indices calculated: mean, 99th quantile, 99% quantile range
#     `years` define the window over which all outputs are pooled.
#     """

#     def read_and_process(metric_id, model, ssp, years, lat, lon):
#         # Read
#         ds_tmp = xr.open_dataset(
#             f"{project_data_path}/metrics_regridded/STAR-ESDM/conservative/{metric_id}_{model}_{ssp}.nc"
#         )
#         ds_tmp["time"] = ds_tmp["time"].dt.year

#         # Time slice
#         ds_sel = ds_tmp.sel(time=slice(years[0], years[1]))

#         # Location selection
#         if lon < 0:
#             lon = 360 + lon
#         ds_sel = ds_sel.sel(lat=lat, lon=lon, method="nearest")

#         # Construct dataframe
#         df_tmp = (
#             ds_sel.to_dataframe().drop(columns=["lat", "lon"]).reset_index()
#         )
#         df_tmp["ssp"] = ssp
#         df_tmp["model"] = model

#         # Return
#         return df_tmp

#     # Check if done
#     if not os.path.isfile(f"{out_path}/{out_str}.csv"):
#         df_delayed = []
#         # Read all
#         for gcm in gcms:
#             df_tmp = dask.delayed(read_and_process)(
#                 metric_id, gcm, ssp, years, lat, lon
#             )
#             df_delayed.append(df_tmp)

#         # Compute and store
#         df_out = dask.compute(*df_delayed)
#         pd.concat(df_out).to_csv(f"{out_path}/{out_str}.csv", index=False)

In [ ]:
# %%time
# for city in city_list.keys():
#     lat, lon = city_list[city]
#     for years in [[2020, 2040], [2050, 2070], [2080, 2100]]:
#         for ssp in ["ssp245", "ssp585"]:
#             for metric_id in [
#                 "avg_tas",
#                 "sum_pr",
#                 "max_tasmax",
#                 "max_pr",
#                 "max_tas",
#             ]:
#                 get_raw_data(
#                     metric_id=metric_id,
#                     ssp=ssp,
#                     years=years,
#                     lat=lat,
#                     lon=lon,
#                     out_path=f"{project_data_path}/summary_raw",
#                     out_str=f"{city}_STAR-ESDM_{ssp}_{str(years[0])}-{str(years[1])}_{metric_id}",
#                 )